## **1.3 NumPy数组运算：通用函数**

### **1.3.1 缓慢的循环**

Python的缓慢通常表现在有很多小操作需要不断的重复，比如对数组中的每个元素做循环操作。假设有一个数组，我们想要计算每个元素的倒数，一种直接的解决方法是：

In [1]:
import numpy as np

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0/values[i]
    return output

values = np.arange(1, 6)
compute_reciprocals(values)

array([1.        , 0.5       , 0.33333333, 0.25      , 0.2       ])

如果测试的数据集很大时，上述操作将非常耗时。我们用`%timeit`函数来测量：

In [2]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

909 ms ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### **1.3.2 通用函数介绍**

对于许多操作，NumPy都为这种静态类型提供了编译好的函数，称为向量化的操作。向量化操作可以简单应用在数组上，实际上会应用在每一个元素上。其实现原理是将循环的部分推送至NumPy之下的编译层，从而提高性能。

In [3]:
print(compute_reciprocals(values))
print(1.0/values)

[1.         0.5        0.33333333 0.25       0.2       ]
[1.         0.5        0.33333333 0.25       0.2       ]


如果计算一个大型数组的运算时间，向量化操作比Python循环花费的时间更短：

In [4]:
# Python循环
%timeit compute_reciprocals(big_array)

1.15 s ± 267 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# 向量操作
%timeit (1.0/big_array)

2.03 ms ± 62.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


NumPy中的向量化操作是通过通用函数（ufunc）实现的，通用函数的主要目的是在NumPy数组中快速执行重复的元素操作，它非常灵活。之前我们展示了标量和数组的运算，其实也可以对两个数组进行运算：

In [6]:
np.arange(5)/np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

通用函数并不仅限于一维数组的运算，也可以进行多维数组的运算：

In [7]:
arr = np.arange(9).reshape((3, 3))
arr

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [8]:
2**arr

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]], dtype=int32)

使用通用函数进行向量化计算几乎总是比用Python循环实现的计算更加高效，尤其是当数组很大时。

### **1.3.3 探索NumPy的通用函数**

通用函数用两种类型：

- 一元通用函数对单个输入值进行操作
- 二元通用函数对两个输入值进行操作

#### **数组运算**

NumPy通用函数的使用方法非常自然，因为它用到了Python原生的算术运算符。标准的加、减、乘、除都可以使用：

In [9]:
arr1 = np.arange(1, 8)
arr1

array([1, 2, 3, 4, 5, 6, 7])

In [10]:
# 加、减、乘、除
print('arr1 = ', arr1)
print('arr1+5  = ', arr1+5)
print('arr1-5  = ', arr1-5)
print('arr1*2  = ', arr1*2)
print('arr1/2  = ', arr1/2)
# 地板除法运算（只取整数部分）
print('arr1//2 = ', arr1//2)

arr1 =  [1 2 3 4 5 6 7]
arr1+5  =  [ 6  7  8  9 10 11 12]
arr1-5  =  [-4 -3 -2 -1  0  1  2]
arr1*2  =  [ 2  4  6  8 10 12 14]
arr1/2  =  [0.5 1.  1.5 2.  2.5 3.  3.5]
arr1//2 =  [0 1 1 2 2 3 3]


还有`-`（取负运算符）、`**`（指数运算符）和`%`（模运算符）这些一元通用函数：

In [11]:
print('arr1 = ', arr1)
# 取负运算
print('-arr1 = ', -arr1)
# 指数运算
print('arr1**2 = ', arr1**2)
# 取模运算
print('arr1%3 = ', arr1%3)

arr1 =  [1 2 3 4 5 6 7]
-arr1 =  [-1 -2 -3 -4 -5 -6 -7]
arr1**2 =  [ 1  4  9 16 25 36 49]
arr1%3 =  [1 2 0 1 2 0 1]


当然，我们也可以将这些算术运算符组合使用（需要考虑这些运算符的优先级）：

In [12]:
-(0.5*arr1+1)**2

array([ -2.25,  -4.  ,  -6.25,  -9.  , -12.25, -16.  , -20.25])

所有这些算术运算符都是NumPy内置函数的简单封装。例如，`+`运算符就是一个`add`函数的封装：

In [13]:
np.add(arr1, 5)

array([ 6,  7,  8,  9, 10, 11, 12])

下表列出了所有NumPy实现的算术运算符：

| 运算符 | 对应的通用函数 | 说明 |
|--------|---------------|--------------------------|
| +      | np.add           | 加法（例如 1+1=2）   |
| -      | np.subtract      | 减法（例如 3-2=1）   |
| -      | np.negative      | 一元取负（例如 -2）      |
| *      | np.multiply      | 乘法（例如 2*3=6）   |
| /      | np.divide        | 除法（例如 3/2=1.5） |
| //     | np.floor_divide  | 整除（例如 3//2=1）  |
| **     | np.power         | 求幂（例如 2**3=8）  |
| %      | np.mod           | 取模（例如 9%4=1）   |

除此之外还有布尔和二进制位操作，我们将在[比较、掩码和布尔逻辑](./1.6%20比较、掩码和布尔逻辑.ipynb)中进行介绍。

#### **绝对值**

正如NumPy可理解Python内置的算术运算操作，NumPy也可以理解Python内置的绝对值函数：

In [14]:
arr2 = np.arange(-10, 1)
arr2

array([-10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0])

In [15]:
abs(arr2)

array([10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

对应的NumPy通用函数是`np.absolute`，该函数还有一个简写形式`np.abs`：

In [16]:
np.abs(arr2)

array([10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [17]:
np.absolute(arr2)

array([10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

该通用函数也可以处理复数。当处理复数时，绝对值返回的是该复数的模：

In [18]:
arr2 = np.array([3-4j, 4-3j, 2+3j, 6+8j])
np.abs(arr2)

array([ 5.        ,  5.        ,  3.60555128, 10.        ])

#### **三角函数**

NumPy提供了大量有用的通用函数，对于数据科学家来说非常有用的还包括三角函数。我们首先定义一个角度数组：

In [19]:
theta = np.linspace(0, np.pi, 3)
theta

array([0.        , 1.57079633, 3.14159265])

现在可以对这些值进行一些三角函数计算：

In [20]:
print('theta      = ', theta)
print('sin(theta) = ', np.sin(theta))
print('cos(theta) = ', np.cos(theta))
print('tan(theta) = ', np.tan(theta))

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


计算得到的值受到计算机浮点数精度的限制，所以有些应该是0的值并不精确地等于0。逆三角函数同样可以使用：

In [21]:
# 逆三角函数同样可以使用
arr3 = np.array([-1, 0, 1])
print('arr1         = ', arr3)
print('arcsin(arr1) = ', np.arcsin(arr3))
print('arccos(arr1) = ', np.arccos(arr3))
print('arctan(arr1) = ', np.arctan(arr3))

arr1         =  [-1  0  1]
arcsin(arr1) =  [-1.57079633  0.          1.57079633]
arccos(arr1) =  [3.14159265 1.57079633 0.        ]
arctan(arr1) =  [-0.78539816  0.          0.78539816]


#### **指数和对数**

NumPy中另一个常用的操作是指数运算：

In [22]:
arr4 = np.arange(1, 5)
print('arr1 = ', arr4)
print('e^x  = ', np.exp(arr4))
print('2^x  = ', np.exp2(arr4))
print('3^x  = ', np.power(3, arr4))

arr1 =  [1 2 3 4]
e^x  =  [ 2.71828183  7.3890561  20.08553692 54.59815003]
2^x  =  [ 2.  4.  8. 16.]
3^x  =  [ 3  9 27 81]


指数运算的逆运算，即对数运算也是可用的。最基本的`np.log`是以自然数为底数的对数。以2为底或以10为底可按以下示例处理：

In [23]:
arr5 = np.arange(1, 5)
print('arr2        = ', arr5)
print('ln(arr2)    = ', np.log(arr5))
print('log2(arr2)  = ', np.log2(arr5))
print('log10(arr2) = ', np.log10(arr5))

arr2        =  [1 2 3 4]
ln(arr2)    =  [0.         0.69314718 1.09861229 1.38629436]
log2(arr2)  =  [0.        1.        1.5849625 2.       ]
log10(arr2) =  [0.         0.30103    0.47712125 0.60205999]


还有一些特殊的版本，对于非常小的输入值可以保持较好的精度：

In [24]:
arr6 = np.array([0, 0.1, 0.01, 0.001])
print('exp(arr3) - 1 = ', np.expm1(arr6))
print('log(1 + arr3) = ', np.log1p(arr6))

exp(arr3) - 1 =  [0.         0.10517092 0.01005017 0.0010005 ]
log(1 + arr3) =  [0.         0.09531018 0.00995033 0.0009995 ]


当输入的值很小时，以上函数给出的值比`np.log`和`np.exp`的计算更精确。

#### **专用的通用函数**

除了上述函数，NumPy还提供了很多通用函数，包括双曲三角函数、比特位运算、弧度转换为角度等等，想了解更多可浏览[NumPy的在线文档](https://numpy.org/doc/stable/)。

在`scipy.special`模块中还有一个更加特殊且难懂的通用函数（如果需要使用到晦涩的数学函数来处理相关数据，基本都可以在这个模块中找到）。

In [25]:
from scipy import special

# Gamma函数（广义阶乘）和相关函数
arr1 = np.array([1, 5, 10])
# Gamma函数
print('Gamma(arr1)     = ', special.gamma(arr1))
# Gamma函数的自然对数
print('ln|Gamma(arr1)| = ', special.gammaln(arr1))
# Beta函数（第一类欧拉积分）
print('beta(arr1, 2)   = ', special.beta(arr1, 2))

Gamma(arr1)     =  [1.0000e+00 2.4000e+01 3.6288e+05]
ln|Gamma(arr1)| =  [ 0.          3.17805383 12.80182748]
beta(arr1, 2)   =  [0.5        0.03333333 0.00909091]


In [26]:
# 误差函数（高斯函数积分）
# 互补误差函数，逆误差函数
x = np.array([0, 0.3, 0.7, 1.0])
# 误差函数
print("erf(x)    =", special.erf(x))
# 互补误差函数
print("erfc(x)   =", special.erfc(x))
# 逆误差函数
print("erfinv(x) =", special.erfinv(x))

erf(x)    = [0.         0.32862676 0.67780119 0.84270079]
erfc(x)   = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


`scipy.special`模块中还有很多其它的通用函数，想进一步了解可查询[在线文档](https://docs.scipy.org/doc/scipy/reference/special.html)。

### **1.3.4 高级的通用函数特性**

#### **指定输出**

在进行大量计算时，有时候指定一个用于存放运算结果的数组是非常有用的。不同于创建临时数组，我们可以用这个特性将计算结果直接写入到我们期待的存储位置。所有的通用函数都可以通过`out`参数来指定计算结果的存放位置：

In [27]:
arr1 = np.arange(1, 7)
arr2 = np.empty(6)
np.multiply(arr1, 10, out=arr2)
arr2

array([10., 20., 30., 40., 50., 60.])

输出结果甚至可以指定为数组的视图，例如，我们可以将计算结果隔一个元素写入到一个数组中：

In [28]:
arr3 = np.zeros(12)
np.power(2, arr1, out=arr3[::2])
arr3

array([ 2.,  0.,  4.,  0.,  8.,  0., 16.,  0., 32.,  0., 64.,  0.])

如果这里没有使用`out`参数，而是写成`arr3[::2] = 2**arr1`，那么结果将是创建一个临时数组用来存储`2**arr1`，然后再将这些值复制到arr3数组中

上述例子的计算量比较小，是否使用`out`参数其实差别并不大，但是对于较大的数组，使用`out`参数能节省很多内存空间

#### **聚合**

二元通用函数有些非常有趣的聚合功能，这些聚合可以直接在数组对象上计算。如果我们希望`reduce`一个数组，那么可以对任何通用函数应用`reduce`方法。`reduce`方法会重复在数组的每一个元素进行通用函数操作，直到得到单个结果。

例如，如`add`通用函数调用`reduce`方法会返回数组中所有元素的和：

In [29]:
arr1 = np.arange(1, 6)
# 1+2+3+4+5 = 15
np.add.reduce(arr1)

15

同样，对`multiply`通用函数调用`reduce`方法会返回数组中所有元素的乘积：

In [30]:
# 1*2*3*4*5 = 120
np.multiply.reduce(arr1)

120

如果需要存储每次计算的中间结果，可以使用`accumulate`方法：

In [31]:
np.add.accumulate(arr1)

array([ 1,  3,  6, 10, 15])

In [32]:
np.multiply.accumulate(arr1)

array([  1,   2,   6,  24, 120])

注意，对于以上这种特殊情况，NumPy也提供了相应的函数直接计算结果（`np.sum`、`np.prod`、`np.cumsum`、`np.cumprod`），我们会在[聚合：最小值、最大值和其他值](./1.4%20聚合：最小值、最大值和其他值.ipynb)中进行详细介绍。

#### **外积**

最后，任何通用函数都可以使用`outer`方法计算所有两个不同输入对的函数运算结果。这意味着我们可以用一行代码实现一个乘法表：

In [33]:
arr1 = np.arange(1, 10)
np.multiply.outer(arr1, arr1)

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 2,  4,  6,  8, 10, 12, 14, 16, 18],
       [ 3,  6,  9, 12, 15, 18, 21, 24, 27],
       [ 4,  8, 12, 16, 20, 24, 28, 32, 36],
       [ 5, 10, 15, 20, 25, 30, 35, 40, 45],
       [ 6, 12, 18, 24, 30, 36, 42, 48, 54],
       [ 7, 14, 21, 28, 35, 42, 49, 56, 63],
       [ 8, 16, 24, 32, 40, 48, 56, 64, 72],
       [ 9, 18, 27, 36, 45, 54, 63, 72, 81]])

`ufunc.at`和`ufunc.reduceat`方法也非常有用，我们会在[花哨的索引](./1.7%20花哨的索引.ipynb)中进行介绍。

通用函数还有一个很有用的特性，能让通用函数在不同长度和形状的数组之间进行计算，这种方法被称为广播。这是一个非常重要的内容，我们会专门在[数组的计算：广播](./1.5%20数值的计算：广播.ipynb)小节中进行介绍。

### **1.3.5 通用函数：更多信息**

有关通用函数的更多信息（包括可用的通用函数的完整列表）可以在[在线文档](https://numpy.org/doc/stable/reference/ufuncs.html)中找到。